To run this colab, you firs need to upload the harry potter dataset into your google drive.

# Preparation

In [ ]:
from google.colab import drive
from google.colab import files
import numpy as np
from scipy.stats import zscore
from scipy import stats

import nltk
nltk.download('punkt')

import torch
import transformers
#from transformers import AutoModelForSequenceClassification
from transformers import AutoModel, AutoModelForMaskedLM
from transformers import AutoTokenizer
#from tokenizers import Regex, Sequence
#from tokenizers.pre_tokenizers import Split


from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import seaborn

In [ ]:
pretrained_weights = 'gpt2'
#pretrained_weights = 'distilbert-base-uncased'
model = AutoModel.from_pretrained(pretrained_weights,
                                  output_hidden_states=True,
                                  output_attentions=True)
#nopunct = Split(Regex("\p{Punctuation}"), "removed")
if pretrained_weights in ['gpt2', 'gpt2-medium', 'gpt2-large']:
    tokenizer = AutoTokenizer.from_pretrained(pretrained_weights, use_fast=True, add_prefix_space=True)
    tokenizer.pad_token = tokenizer.eos_token
else:
    tokenizer = AutoTokenizer.from_pretrained(pretrained_weights, use_fast=True)

#existing = tokenizer.backend_tokenizer.pre_tokenizer
#tokenizer.backend_tokenizer.pre_tokenizer = Sequence([nopunct, existing])

In [ ]:
#@title Mount google drive (where the data is uploaded).

drive.mount('/content/drive')

In [ ]:
#base_dir = '/srv/data/uva/fmri/HP_data/fMRI'
base_dir = '/content/drive/My Drive/HP_fmri'
subject_id = 'K' #@param 
subject_data = np.load(f'{base_dir}/data_subject_{subject_id}.npy')
time_fmri = np.load(f'{base_dir}/time_fmri.npy')
runs_fmri = np.load(f'{base_dir}/runs_fmri.npy')
words_fmri = np.load(f'{base_dir}/words_fmri.npy')
time_words_fmri = np.load(f'{base_dir}/time_words_fmri.npy')

In [ ]:
words_fmri.shape

In [ ]:
SKIP_WORDS = 20
END_WORDS = 5129

In [ ]:
def split_sentence_naive(words):
    sent = []
    for word in words:
#        print(word, sent)
        if word == ".":
            continue
        if word[0] == '"':
            word = word[1:]
        if word[-1] == '"':
            word = word[:-1]
        if word[-1] in [".", "!", "?"]:
            sent.append(word[:-1])
#            print("YIELD")
            yield sent
            sent = []
        elif word[-1] == ",":
            sent.append(word[:-1])
        else:
            sent.append(word)
    if len(sent) > 0:
#        print("YIELD")
        yield sent

In [ ]:
naive_sents = list(split_sentence_naive(words_fmri))

In [ ]:
encoded_sentences = tokenizer.batch_encode_plus(naive_sents, is_split_into_words = True, padding=True, return_tensors="pt")

print('sentence:', naive_sents[0])
print('encoded sentences:', encoded_sentences['input_ids'][0], encoded_sentences['attention_mask'][0])

In [ ]:
class SentenceDataset(torch.utils.data.Dataset):
    """
        Custom Dataset class that lets us return the word ID list as
        a tensor so it is compatible with the DataLoader. We need the
        word IDs to merge back together the words that get split into
        multiple tokens by the tokenizer
    """
    def __init__(self, sents:transformers.tokenization_utils_base.BatchEncoding):
        self.sents = sents
    def __len__(self):
        return self.sents['input_ids'].size(0)
    def __getitem__(self, idx:int):
        sent = self.sents[idx]
#        print("input_ids", sent.ids)
#        print("word_ids", sent.word_ids)
#        print("mask", sent.attention_mask)
        return {
            'input_ids': torch.tensor(sent.ids),
            'word_ids': torch.tensor(list(map(lambda e: -1 if e is None else e, sent.word_ids))),
            'attention_mask': torch.tensor(sent.attention_mask)
        }

In [ ]:
def get_batches(input_dict, batch_size :int = 2):
    tensor_dataset = SentenceDataset(input_dict)
    tensor_dataloader = torch.utils.data.DataLoader(tensor_dataset, batch_size=batch_size)
    
    return tensor_dataloader

In [ ]:
dl = get_batches(encoded_sentences, batch_size=11)

In [ ]:
def get_word_vectors(model, inputs, attention_mask, states, word_ids, layers):
    """ Push input IDs through model. Stack and sum `layers` (last four by default).
        Select only those subword token outputs that belong to our word of interest
        and average them.""" 
    token_len = attention_mask.sum().item()
    if pretrained_weights in ['gpt2', 'gpt2-medium', 'gpt2-large']:
        word_indices = np.array(list(map(lambda e: -1 if e is None else e, word_ids)))[:token_len]
#        print(word_indices)
        word_groups = np.split(np.arange(word_indices.shape[0]), np.unique(word_indices, return_index=True)[1])[1:]
#        print(word_groups)
    else:
        word_indices = np.array(list(map(lambda e: -1 if e is None else e, word_ids)))[1:token_len - 1]
        word_groups = np.split(np.arange(word_indices.shape[0]) + 1, np.unique(word_indices, return_index=True)[1])[1:]
    # Stack all words in the sentence
    sent_tokens_output = torch.stack([
        # Sum the requested layers
        torch.stack([
                states[i].detach()[token_ids_word].mean(axis=0)
                    if i > 0 else
                model.embeddings.word_embeddings(inputs)[token_ids_word].mean(axis=0)
                        for i in layers
            ]).sum(axis=0).squeeze()
                for token_ids_word in word_groups
        ])
    return sent_tokens_output


In [ ]:
nlp_features = None
for batch, input_dict in enumerate(dl):
    word_ids = input_dict.pop('word_ids')
    mask = input_dict['attention_mask']
    input_ids = input_dict['input_ids']
    output = model(**input_dict)

    activations = torch.transpose(torch.stack(output.hidden_states), 0, 1)
    result = np.concatenate(
            [
                get_word_vectors(model, si, sm, sa, sw, [1])
                    for sw, sm , si, sa in zip(word_ids, mask, input_ids, activations)
                            if pretrained_weights in ['gpt2', 'gpt2-medium', 'gpt2-large'] or sm.sum().item() > 2

            ]
        )
    if nlp_features is None:
        nlp_features = result
    else:
        nlp_features = np.concatenate([nlp_features, result])
print(nlp_features.shape)

In [ ]:
def delay_one(mat, d):
    """delays a matrix by a delay d. Positive d ==> row t has row t-d."""
    new_mat = np.zeros_like(mat)
    if d>0:
        new_mat[d:] = mat[:-d]
    elif d<0:
        new_mat[:d] = mat[-d:]
    else:
        new_mat = mat
    return new_mat

def delay_mat(mat, delays):
    """ delays a matrix by a set of delays d.
        a row t in the returned matrix has the concatenated:
        {row(t-delays[0],t-delays[1]...t-delays[last] ).
    """
    new_mat = np.concatenate([delay_one(mat, d) for d in delays],axis = -1)
    return new_mat

def prepare_nlp_features(train_features, test_features, word_train_indicator, TR_train_indicator, SKIP_WORDS=20, END_WORDS=5129):
        
    time = np.load(f'{base_dir}/time_fmri.npy')
    runs = np.load(f'{base_dir}/runs_fmri.npy') 
    time_words = np.load(f'{base_dir}/time_words_fmri.npy')
    time_words = time_words[SKIP_WORDS:END_WORDS]
        
    words_id = np.zeros([len(time_words)])
    # w=find what TR each word belongs to
    for i in range(len(time_words)):
        words_id[i] = np.where(time_words[i]> time)[0][-1]
        
    all_features = np.zeros([time_words.shape[0], train_features.shape[1]])
    all_features[word_train_indicator] = train_features
    all_features[~word_train_indicator] = test_features
        
    p = all_features.shape[1]
    tmp = np.zeros([time.shape[0], p])
    for i in range(time.shape[0]):
        tmp[i] = np.mean(all_features[(words_id<=i)*(words_id>i-1)],0)
    tmp = delay_mat(tmp, np.arange(1,5))

    # remove the edges of each run
    tmp = np.vstack([zscore(tmp[runs==i][20:-15]) for i in range(1,5)])
    tmp = np.nan_to_num(tmp)
        
    return tmp[TR_train_indicator], tmp[~TR_train_indicator]

def get_fold_flags(n, n_folds):
    flags = np.zeros((n))
    num_items_in_each_fold = int(np.floor(n/n_folds))
    for i in range(0,n_folds -1):
        flags[i*num_items_in_each_fold:(i+1)*num_items_in_each_fold] = i
    flags[(n_folds-1)*num_items_in_each_fold:] = (n_folds-1)
    return flags

def tr_to_word_train_indicator(tr_train_indicator, skip_words=20, end_words=5176):
    time = np.load(f'{base_dir}/time_fmri.npy')
    runs = np.load(f'{base_dir}/runs_fmri.npy') 
    time_words = np.load(f'{base_dir}/time_words_fmri.npy')
    time_words = time_words[skip_words:end_words]
        
    word_train_indicator = np.zeros([len(time_words)], dtype=bool)    
    words_id = np.zeros([len(time_words)],dtype=int)
    # Find what TR each word belongs to.
    for i in range(len(time_words)):                
        words_id[i] = np.where(time_words[i]> time)[0][-1]
        
        if words_id[i] <= len(runs) - 15:
            offset = runs[int(words_id[i])]*20 + (runs[int(words_id[i])]-1)*15
            if tr_train_indicator[int(words_id[i])-offset-1] == 1:
                word_train_indicator[i] = True
    return word_train_indicator 

In [ ]:
n_words = subject_data.shape[0]
n_voxels = subject_data.shape[1]
print(f'Number of voxels for this subject is {n_voxels} for {n_words} scans.')

In [ ]:
n_folds = 4
test_fold = 1
skip = 5
fold_flags = get_fold_flags(n_words, n_folds=n_folds)
train_ind = fold_flags!=test_fold
test_ind = fold_flags==test_fold
train_indicator = tr_to_word_train_indicator(train_ind, skip_words=SKIP_WORDS, end_words=END_WORDS)

In [ ]:
nlp_features.shape

In [ ]:
train_nlp_features = nlp_features[SKIP_WORDS:END_WORDS,:][train_indicator]
test_nlp_features = nlp_features[SKIP_WORDS:END_WORDS,:][~train_indicator]

In [ ]:
train_features, test_features = prepare_nlp_features(
        train_nlp_features,
        test_nlp_features,
        train_indicator,
        train_ind,
        SKIP_WORDS=SKIP_WORDS,
        END_WORDS=END_WORDS
    )

In [ ]:
print(train_features.shape, test_features.shape)

In [ ]:
#@title Get fmri data points.

train_data = subject_data[train_ind]
test_data = subject_data[test_ind]

In [ ]:
#@title Filter out some of data points in the margins of test and train sets.

# skip TRs between train and test data
if test_fold == 0: # just remove from front end
    train_data = train_data[skip:,:]
    train_features = train_features[skip:,:]
elif test_fold == n_folds-1: # just remove from back end
    train_data = train_data[:-skip,:]
    train_features = train_features[:-skip,:]
else:
    test_data = test_data[skip:-skip,:]
    test_features = test_features[skip:-skip,:]

# normalize data
train_data = np.nan_to_num(zscore(np.nan_to_num(train_data)))
test_data = np.nan_to_num(zscore(np.nan_to_num(test_data)))

In [ ]:
print('fmri features:', train_data.shape, test_data.shape)
print('nlp features:', train_features.shape, test_features.shape)

# Representational Similarity

In [ ]:
from itertools import combinations
from functools import partial
import mantel
def my_pearson_rsa(DM1, DM2):
    """Compute representational similarity between two disimilarity matrices
    """
    # selection elements of the upper triangle
    elements1 = DM1[np.triu_indices(DM1.shape[1],k=1)]
    elements2 = DM2[np.triu_indices(DM2.shape[1],k=1)]

    # Here you can do pearson-r instead of this (or again any measure of similarity).
    correlation_of_similarities = stats.pearsonr(elements1,elements2)

    #print("upper triangular1:",elements1)
    #print("upper triangular2:",elements2)
    return correlation_of_similarities

mantel_func = lambda l, r: mantel.test(l, r, method="pearson")[0]


In [ ]:
def dot_product_rsa(reps1, reps2):
    """Compute representational similarity between two sets of representations.

       Args:
            reps1: float array; [num_examples, feature_size1]
            reps2: float array; [num_examples, feature_size2]
    """
    assert reps1.shape[0] == reps2.shape[0], 'First dimensions of inputs should match.'

    # Normalize input representations
    reps1 = reps1 / np.linalg.norm(reps1, axis=-1)[...,None]
    reps2 = reps2 / np.linalg.norm(reps2, axis=-1)[...,None]

    # Compute and normalize similarity matrices:
    similarities1 = reps1.dot(reps1.T) # instead of dot product you can use any similarity measure
    similarities2 = reps2.dot(reps2.T)
    similarities1 = similarities1 / np.linalg.norm(similarities1, axis=-1)[...,None]
    similarities2 = similarities2 / np.linalg.norm(similarities2, axis=-1)[...,None]

    # Here you can do pearson-r instead of this (or again any measure of similarity).
    similarity_of_similarity = np.sum(similarities1 * similarities2, axis=-1)
    return np.mean(similarity_of_similarity)

In [ ]:
def pearson_rsa(reps1, reps2):
    """Compute representational similarity between two sets of representations.

    Args:
        reps1: float array; [num_examples, feature_size1]
        reps2: float array; [num_examples, feature_size2]
    """
    assert reps1.shape[0] == reps2.shape[0], 'First dimensions of inputs should match.'

    # Normalize input representations
    reps1 = reps1 / np.linalg.norm(reps1, axis=-1)[...,None]
    reps2 = reps2 / np.linalg.norm(reps2, axis=-1)[...,None]

    # Compute and normalize similarity matrices:
    similarities1 = reps1.dot(reps1.T) # instead of dot product you can use any similarity measure
    similarities2 = reps2.dot(reps2.T)
    similarities1 = similarities1 / np.linalg.norm(similarities1, axis=-1)[...,None]
    similarities2 = similarities2 / np.linalg.norm(similarities2, axis=-1)[...,None]

    # Here you can do pearson-r instead of this (or again any measure of similarity).
    correlation_of_similarities = [stats.pearsonr(x,y)[0] for x,y in zip(similarities1,similarities2)]
    return np.mean(correlation_of_similarities)

In [ ]:
def dot_product_rsa_for_lists_of_reps(reps):
    """Compute representational similarity between two sets of representations.

    Args:
        reps: list of float arrays; List of arrrays with shape [num_examples, feature_size]
        (feature size can be different for each item in the list).
    """
    norm_reps = []
    for rep in reps:
        # Normalize input representations
        norm_reps.append(rep / np.linalg.norm(rep, axis=-1)[...,None])

    # Compute and normalize similarity matrices:
    similarity_matrices = []
    similarity_matrices_n = []
    for rep, norm_rep in zip(reps, norm_reps):
        similarities = rep.dot(rep.T) # instead of dot product you can use any similarity measure
        similarity_matrices.append(similarities)
        similarities_n = norm_rep.dot(norm_rep.T) # instead of dot product you can use any similarity measure
        similarities_n = similarities / np.linalg.norm(similarities, axis=-1)[...,None]
        similarity_matrices_n.append(similarities_n)

    # Shape of similarity_matrices: [num_of_rep_spaces, num_examples, num_examples]
    similarity_matrices = np.asarray(similarity_matrices) 
    similarity_matrices_n = np.asarray(similarity_matrices_n)
    sim_of_sim_mat = np.zeros((similarity_matrices.shape[0], 
                               similarity_matrices.shape[0]))
    sim_of_sim_mat2 = np.zeros((similarity_matrices.shape[0], 
                               similarity_matrices.shape[0]))
    sim_of_sim_mat3 = np.zeros((similarity_matrices.shape[0], 
                               similarity_matrices.shape[0]))
    for i in np.arange(sim_of_sim_mat.shape[0]):
        for j in np.arange(sim_of_sim_mat.shape[1]):
            # Here you can do pearson-r instead of this (or again any measure of similarity).
            similarity_of_similarity = np.sum(
                    similarity_matrices_n[i] * similarity_matrices_n[j],
                    axis=-1
                )
            similarity_score = np.mean(similarity_of_similarity)
            sim_of_sim_mat[i][j] = similarity_score
            alt = my_pearson_rsa(similarity_matrices[i], similarity_matrices[j])
            sim_of_sim_mat2[i][j] = alt[0]
            sim_of_sim_mat3[i][j] = alt[1]
    return sim_of_sim_mat, sim_of_sim_mat2, sim_of_sim_mat3

In [ ]:
dot_product_rsa(train_data, train_data)

In [ ]:
pearson_rsa(train_data, train_data)

In [ ]:
dot_product_rsa_for_lists_of_reps([train_data, train_data])

In [ ]:
a, b, c = dot_product_rsa_for_lists_of_reps([train_data, train_features])
print("#> a, b, c = dot_product_rsa_for_lists_of_reps([train_data, train_features])\n")
print("\toriginal\n\n", a, "\n")
print("\t'my_pearson_rsa'\n\n", b, "\n")
print("\tp-values\n\n", c, "\n")

In [ ]:
dot_product_rsa_for_lists_of_reps([train_features, train_features])

In [ ]:
#@title Compare two subjects:

subject_id_1 = 'K' #@param 
subject_data_1 = np.load(f'{base_dir}/data_subject_{subject_id_1}.npy')

subject_id_2 = 'F' #@param 
subject_data_2 = np.load(f'{base_dir}/data_subject_{subject_id_2}.npy')

subject_data_1 = np.nan_to_num(zscore(np.nan_to_num(subject_data_1)))
subject_data_2 = np.nan_to_num(zscore(np.nan_to_num(subject_data_2)))

print(f'Representational similarity between subject {subject_id_1} and {subject_id_2} is:', dot_product_rsa(subject_data_1, subject_data_2))

In [ ]:
subjects = ['K', 'F', 'J', 'M', 'N']
subject_data_dict = {}

for subject in subjects:
    subject_data_dict[subject] = np.load(f'{base_dir}/data_subject_{subject}.npy')

results = dot_product_rsa_for_lists_of_reps(subject_data_dict.values())
for heading, result in zip(["### original", "### my_pearson_rsa", "### p-values"], results):
    np.fill_diagonal(result, 0)
    display(Markdown(heading))
    display(result)
#    result = result.round(3)
    # put zeros on the diagonal to make the differences between the values stand out more
    # this exaggerates the differences
    np.fill_diagonal(result, 0)
    seaborn.heatmap(data=result, cmap=seaborn.color_palette("coolwarm", as_cmap=True))
    plt.show()